In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

# Function for scraping from indeed

In [ ]:
def scraping(url): 
        j=[]
        s=[]

        for i in range(0,4):#for each page
                page = requests.get(url+'&start='+str(i)+'0')
                time.sleep(2)
                soup = BeautifulSoup(page.content)
        
                for item in soup.find_all('div',attrs={'class':'row'}):#for each job posting
                        job_title = soup.find_all("div", {"class":"title"})
                        for title in job_title:
                            j.append(title.text.strip())
                    
                        skills_data = soup.find_all("div", {"class":"summary"})
                        for sk in skills_data:
                            s.append(sk.text.strip())
        ziplist=  list(zip(j,s))                    
        return ziplist                     
                           

# Function for cleaning the text

In [ ]:
import re
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """  text: a string 
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    
    return text

# Collecting the scraped data into the data frame

In [ ]:
d=scraping('https://www.indeed.co.in/jobs?q=software+Programmer&l=india')
df=pd.DataFrame(data=d,columns=['Job','Skills'])

df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)


df['Skills']=df['Skills'].apply(clean_text)
df['Label']='Software Programmer'
df

In [ ]:
df_sp=df

In [ ]:
df_ac=df

In [ ]:
df_ml=df

In [ ]:
final_df=pd.concat([df_ml,df_ac,df_sp])

In [ ]:
final_df

In [ ]:
df=final_df

In [ ]:
df.to_csv('final_result_ml_sp_ac.csv')

Reading the data into a data frame

In [2]:
df=pd.read_csv('final_result_ml_sp_ac.csv')
df

,Job,Skills,Label
0,Software Engineer,brand rapidly growing time add production team...,Machine Learning
1,Data Engineer,brand rapidly growing time add production team...,Machine Learning
2,[ AWS Sr. Developer/Architect]-Senior Software...,create technical components designed solution ...,Machine Learning
3,Machine Learning Engineer,run machine learning tests experimentsdevelop ...,Machine Learning
4,Machine Learning Engineer - Remote for a UK AI...,first two machine learning engineers supportin...,Machine Learning
...,...,...,...
169,"Software Engineers, System Analysts, Software ...",software engineers system analysts software de...,Software Programmer
170,Electrical Engineer,handson experience using design calculation so...,Software Programmer
171,Senior Statistical Programmer,coordinate activities programmers either inter...,Software Programmer
172,Java with Microservices,experience systems analysis programming softwa...,Software Programmer


# Input Formatting

In [3]:
#Creating word index representation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Skills'].values)
X = tokenizer.texts_to_sequences(df['Skills'].values)

word_index=tokenizer.word_index
print(f'Found {len(word_index)}of unique tokens')

X= pad_sequences(X, padding = 'post') #Zero padding at the end of the sequence

Using TensorFlow backend.
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

Found 1217of unique tokens


In [4]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

l=df['Label'].tolist()
le=preprocessing.LabelEncoder()
Y=le.fit_transform(l)
Y=to_categorical(Y)

In [5]:
print(f'Shape of input:{X.shape}')
print(f'Shape of label:{Y.shape}')

Shape of input:(174, 22)
Shape of label:(174, 3)


# Splitting into training and validation set

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0)

# Downloading pre-trained embedding model

In [7]:
import gensim.downloader
w2v_model= gensim.downloader.load("glove-wiki-gigaword-100") 

# Create embedding layer weights

In [8]:
embedding_matrix=np.zeros((len(word_index)+1,100))

for word,i in word_index.items():
    try:
        embedding_vector=w2v_model[word]
        embedding_matrix[i]=embedding_vector
        
    except KeyError:
        pass
print(f'The shape of embedding matrix is{embedding_matrix.shape}')


The shape of embedding matrix is(1218, 100)


# Create Neural Network

In [9]:
maxlen=len(X[0])
maxlen

22

In [10]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dropout,Dense,Bidirectional

model=Sequential()
model.add(
    Embedding(
              input_dim=len(word_index)+1,
              output_dim=100,
              weights=[embedding_matrix],
              input_length=maxlen,
              trainable=True  
              )
      )

model.add(Bidirectional(LSTM(100)))
model.add(Dense(3,activation='softmax'))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 100)           121800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 603       
Total params: 283,203
Trainable params: 283,203
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Training the model

In [13]:
model.fit(X_train,y_train,
      batch_size=32,
      epochs=4,
      validation_data=(X_test,y_test),
      shuffle=True
    )


Train on 130 samples, validate on 44 samples
Epoch 1/4
130/130 [==============================] - 2s 19ms/step - loss: 1.0928 - accuracy: 0.4077 - val_loss: 0.9609 - val_accuracy: 0.6818
Epoch 2/4
130/130 [==============================] - 0s 3ms/step - loss: 0.8656 - accuracy: 0.8154 - val_loss: 0.8394 - val_accuracy: 0.6818
Epoch 3/4
130/130 [==============================] - 0s 2ms/step - loss: 0.6658 - accuracy: 0.8615 - val_loss: 0.6021 - val_accuracy: 0.8409
Epoch 4/4
130/130 [==============================] - 0s 3ms/step - loss: 0.4637 - accuracy: 0.8615 - val_loss: 0.3859 - val_accuracy: 0.8864


# Evaluating the model

In [14]:
score=model.evaluate(X_test,y_test,verbose=True,batch_size=32)
print(f'Accuracy:{score}')

44/44 [==============================] - 0s 709us/step
Accuracy:[0.3858863901008259, 0.8863636255264282]


In [15]:
score=model.evaluate(X_train,y_train,verbose=True,batch_size=32)
print(f'Accuracy:{score}')

130/130 [==============================] - 0s 840us/step
Accuracy:[0.3520195511671213, 0.8999999761581421]


In [16]:
y_pred=model.predict_classes(X_test)
y_pred.shape

(44,)

In [17]:
y_test=np.argmax(y_test,axis=-1)

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[15,  0,  0],
       [ 0, 14,  2],
       [ 0,  3, 10]], dtype=int64)

# Output

In [19]:
p=model.predict_classes(
       pad_sequences(
         tokenizer.texts_to_sequences(['The candidate needs to manage all financial transactions, from fixed payments and variable expenses to bank deposits Manage all accounting transactions',
                                       'Hands on experience in Image Processing and Pattern Recognition. Solid Python Development skills and hands on experience in two or more of the following packages - Keras, Tensorflow, PyTorch, Ludwig, TensorRT, SciPy and NumPy',
                                       'Strong background in learning algorithms (multi-class classifications, decision trees and deep learning)','neural network,deep learning'
                                       ]),maxlen=22,padding='post'))
le.inverse_transform(p)

array(['Accountant', 'Machine Learning', 'Machine Learning',
       'Machine Learning'], dtype='<U19')

In [20]:
p=model.predict_classes(
       pad_sequences(
         tokenizer.texts_to_sequences(['good communication skills and expert in python'
                                       ]),maxlen=maxlen,padding='post'))
le.inverse_transform(p)

array(['Software Programmer'], dtype='<U19')

In [33]:
p=model.predict_classes(
       pad_sequences(
         tokenizer.texts_to_sequences(['c++'
                                       ]),maxlen=maxlen,padding='post'))
le.inverse_transform(p)

array(['Accountant'], dtype='<U19')